In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import sklearn as skl
import seaborn as sns
import config
from sqlalchemy import create_engine
import math
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError



In [2]:
#set up SQL connection

protocol = 'postgresql'
username = config.username
password = config.password
host = 'localhost'
port = 5432
database_name = config.database_name
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
con = engine.connect()

In [3]:
#read in cleaned X table from SQL

cleaned_placeholder_df = pd.read_sql('Select * from "X"',con)
cleaned_placeholder_df.head()

,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,netProfitMargin,pretaxProfitMargin,grossProfitMargin,operatingProfitMargin,returnOnAssets,returnOnCapitalEmployed,...,Rating Agency Name_Standard & Poor's Ratings Services,Sector_Basic Industries,Sector_Capital Goods,Sector_Consumer Non-Durables,Sector_Consumer Services,Sector_Energy,Sector_Health Care,Sector_Other,Sector_Public Utilities,Sector_Technology
0,0.945894,0.426395,0.099690,44.203245,0.037480,0.049351,0.176631,0.061510,0.041189,0.091514,...,0,0,0,0,0,0,0,1,0,0
1,1.033559,0.498234,0.203120,38.991156,0.044062,0.048857,0.175715,0.066546,0.053204,0.104800,...,0,0,0,0,0,0,0,1,0,0
2,0.963703,0.451505,0.122099,50.841385,0.032709,0.044334,0.170843,0.059783,0.032497,0.075955,...,0,0,0,0,0,0,0,1,0,0
3,1.019851,0.510402,0.176116,41.161738,0.020894,-0.012858,0.138059,0.042430,0.025690,-0.027015,...,0,0,0,0,0,0,0,1,0,0
4,0.957844,0.495432,0.141608,47.761126,0.042861,0.053770,0.177720,0.065354,0.046363,0.096945,...,1,0,0,0,0,0,0,1,0,0


In [4]:
cleaned_placeholder_df.shape

(2029, 39)

In [5]:
#read in cleaned Y table from SQL alternate_y.csv.
cleaned_y_df = pd.read_sql("Select * from alternate_y",con)
cleaned_y_df.head()

,Rating
0,1
1,0
2,0
3,0
4,0


In [6]:
# Split our preprocessed data into our features and target arrays
y = cleaned_y_df['Rating'].values
y

# declare x variable
X = cleaned_placeholder_df.values
X

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 55)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_layer = len(X_train_scaled[0])
hidden_nodes_L1= 60
hidden_nodes_L2 = 30
hidden_nodes_L3 = 10


nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_L1, activation="relu", input_dim=input_layer))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_L2, activation="relu"))

# Third hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_L3, activation="sigmoid"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="selu"))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                2400      
                                                                 
 dense_1 (Dense)             (None, 30)                1830      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 4,551
Trainable params: 4,551
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# Train the model
nn1.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
48/48 [==============================] - 1s 977us/step - loss: 0.5975 - accuracy: 0.6884
Epoch 2/100
48/48 [==============================] - 0s 993us/step - loss: 0.5148 - accuracy: 0.7811
Epoch 3/100
48/48 [==============================] - 0s 861us/step - loss: 0.4922 - accuracy: 0.7955
Epoch 4/100
48/48 [==============================] - 0s 943us/step - loss: 0.4885 - accuracy: 0.7936
Epoch 5/100
48/48 [==============================] - 0s 941us/step - loss: 0.4710 - accuracy: 0.7988
Epoch 6/100
48/48 [==============================] - 0s 936us/step - loss: 0.4717 - accuracy: 0.8028
Epoch 7/100
48/48 [==============================] - 0s 947us/step - loss: 0.4643 - accuracy: 0.8028
Epoch 8/100
48/48 [==============================] - 0s 938us/step - loss: 0.4590 - accuracy: 0.8041
Epoch 9/100
48/48 [==============================] - 0s 991us/step - loss: 0.4635 - accuracy: 0.8047
Epoch 10/100
48/48 [==============================] - 0s 936us/step - loss: 0.4561 - accura

Epoch 82/100
48/48 [==============================] - 0s 916us/step - loss: 0.3306 - accuracy: 0.8481
Epoch 83/100
48/48 [==============================] - 0s 900us/step - loss: 0.3283 - accuracy: 0.8554
Epoch 84/100
48/48 [==============================] - 0s 800us/step - loss: 0.3261 - accuracy: 0.8567
Epoch 85/100
48/48 [==============================] - 0s 872us/step - loss: 0.3357 - accuracy: 0.8554
Epoch 86/100
48/48 [==============================] - 0s 846us/step - loss: 0.3176 - accuracy: 0.8554
Epoch 87/100
48/48 [==============================] - 0s 894us/step - loss: 0.3372 - accuracy: 0.8501
Epoch 88/100
48/48 [==============================] - 0s 890us/step - loss: 0.3329 - accuracy: 0.8560
Epoch 89/100
48/48 [==============================] - 0s 814us/step - loss: 0.3220 - accuracy: 0.8593
Epoch 90/100
48/48 [==============================] - 0s 917us/step - loss: 0.3145 - accuracy: 0.8646
Epoch 91/100
48/48 [==============================] - 0s 854us/step - loss: 0.3154

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

16/16 - 0s - loss: 0.7596 - accuracy: 0.7815 - 153ms/epoch - 10ms/step
Loss: 0.7596246600151062, Accuracy: 0.7814960479736328


In [12]:
# Export our model to HDF5 file
nn1.save("Baseline_model_Credit_rating.h5")

# Using hyper parameter to increase accuracy

In [27]:
import kerastuner as kt
msle = MeanSquaredLogarithmicError()


def build_model(hp):
  model = tf.keras.Sequential()
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
  hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
  hp_units3 = hp.Int('units3', min_value=32, max_value=512, step=32)
  model.add(Dense(units=hp_units1, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units2, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units3, activation='sigmoid'))
  model.add(Dense(1, kernel_initializer='normal', activation='selu'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss=msle,
      metrics=[msle]
  )

  return model

# HyperBand algorithm from keras tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_mean_squared_logarithmic_error',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)

tuner.search(X_train_scaled, y_train, epochs=10, validation_split=0.2)

INFO:tensorflow:Reloading Oracle from existing project keras_tuner_dir\keras_tuner_demo\oracle.json
INFO:tensorflow:Reloading Tuner from keras_tuner_dir\keras_tuner_demo\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [28]:
for h_param in [f"units{i}" for i in range(1,4)] + ['learning_rate']:
  print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

units1 64
units2 288
units3 128
learning_rate 0.001


In [29]:
best_model = tuner.get_best_models()[0]
best_model.build(X_train_scaled.shape)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (1521, 64)                2560      
                                                                 
 dense_1 (Dense)             (1521, 288)               18720     
                                                                 
 dense_2 (Dense)             (1521, 128)               36992     
                                                                 
 dense_3 (Dense)             (1521, 1)                 129       
                                                                 
Total params: 58,401
Trainable params: 58,401
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_layer = len(X_train_scaled[0])
hidden_nodes_L1= 64
hidden_nodes_L2 = 288
hidden_nodes_L3 = 128


nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_L1, activation="relu", input_dim=input_layer))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_L2, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_L3, activation="sigmoid"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="selu"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                2560      
                                                                 
 dense_5 (Dense)             (None, 288)               18720     
                                                                 
 dense_6 (Dense)             (None, 128)               36992     
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 58,401
Trainable params: 58,401
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
48/48 [==============================] - 1s 2ms/step - loss: 0.7228 - accuracy: 0.6956
Epoch 2/100
48/48 [==============================] - 0s 1ms/step - loss: 0.7019 - accuracy: 0.7107
Epoch 3/100
48/48 [==============================] - 0s 1ms/step - loss: 0.8346 - accuracy: 0.7469
Epoch 4/100
48/48 [==============================] - 0s 1ms/step - loss: 0.5817 - accuracy: 0.7942
Epoch 5/100
48/48 [==============================] - 0s 1ms/step - loss: 0.5357 - accuracy: 0.7982
Epoch 6/100
48/48 [==============================] - 0s 1ms/step - loss: 0.5526 - accuracy: 0.7844
Epoch 7/100
48/48 [==============================] - 0s 1ms/step - loss: 0.5405 - accuracy: 0.7942
Epoch 8/100
48/48 [==============================] - 0s 1ms/step - loss: 0.5266 - accuracy: 0.8041
Epoch 9/100
48/48 [==============================] - 0s 1ms/step - loss: 0.5054 - accuracy: 0.8074
Epoch 10/100
48/48 [==============================] - 0s 1ms/step - loss: 0.5038 - accuracy: 0.7870
Epoch 11/

48/48 [==============================] - 0s 2ms/step - loss: 0.3758 - accuracy: 0.8369
Epoch 84/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3648 - accuracy: 0.8422
Epoch 85/100
48/48 [==============================] - 0s 1ms/step - loss: 0.3571 - accuracy: 0.8396
Epoch 86/100
48/48 [==============================] - 0s 1ms/step - loss: 0.3558 - accuracy: 0.8481
Epoch 87/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3439 - accuracy: 0.8527
Epoch 88/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3356 - accuracy: 0.8488
Epoch 89/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3373 - accuracy: 0.8501
Epoch 90/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3630 - accuracy: 0.8416
Epoch 91/100
48/48 [==============================] - 0s 1ms/step - loss: 0.6982 - accuracy: 0.7890
Epoch 92/100
48/48 [==============================] - 0s 1ms/step - loss: 0.4594 - accuracy: 0.8120
Epoch 93/100


In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

16/16 - 0s - loss: 0.7746 - accuracy: 0.7992 - 136ms/epoch - 9ms/step
Loss: 0.7746039628982544, Accuracy: 0.7992125749588013


In [34]:
# Export our model to HDF5 file
nn2.save("Optimized_Credit_rating.h5")